In [1]:
import pandas as pd
import requests

import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import glob
import os

In [2]:
import re

def get_key_from_environment(file_path: str, key: str) -> str | None:
    """
    Extracts a key from Angular's environment.ts file.

    Args:
        file_path: Path to environment.ts
        key: The key name (e.g., "apiKey")

    Returns:
        The value as a string, or None if not found.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Regex to match key: 'value' or key: "value"
    pattern = rf'{key}\s*:\s*[\'"]([^\'"]+)[\'"]'
    match = re.search(pattern, content)

    return match.group(1) if match else None


# Example usage
file_path = "../src/environments/environment.ts"
api_key = get_key_from_environment(file_path, "apiToken")

print("API Key:", api_key)


API Key: 55a064fc7f01134ea1d951c30a610354


In [3]:
header = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [20]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

scale = 12
start_date = datetime(2024, 9, 1)   # Sept 2024
end_date = datetime(2025, 8, 1)     # Aug 2025

# Loop over months
date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m")
    url = f"https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale{scale:03d}&date={date_str}"
    
    print(f"Fetching {url} ...")
    res = requests.get(url, headers=header)
    
    if res.status_code == 200:
        file = f"./data/spi{scale:03d}_{date_str}.tif"
        with open(file, "wb") as f:
            f.write(res.content)
    
    # Move to next month
    date += relativedelta(months=1)

Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2024-09 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2024-10 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2024-11 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2024-12 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2025-01 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2025-02 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2025-03 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2025-04 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale012&date=2025-05 ...
Fetching h

In [21]:
shapefile = "../public/hawaii_climate_divisions.shp"

raster_folder = "./data"

gdf = gpd.read_file(shapefile)

id_col = "name"  

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, f"spi{scale:03d}*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Zonal statistics
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    # Store results
    for div, s in zip(gdf[id_col], stats):
        records.append({
            "division": div,
            "date": date,
            "mean_spi": s["mean"]
        })

df = pd.DataFrame(records)
df = df.pivot(index="division", columns="date", values="mean_spi")
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv(f"../public/division_spi{scale}.csv")



In [22]:
shapefile = "../public/Coastline.shp"

raster_folder = "./data"

gdf = gpd.read_file(shapefile)

id_col = "isle"  

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, f"spi{scale:03d}*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Zonal statistics
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    # Store results
    for div, s in zip(gdf[id_col], stats):
        records.append({
            "division": div,
            "date": date,
            "mean_spi": s["mean"]
        })

df = pd.DataFrame(records)
df = df.pivot(index="division", columns="date", values="mean_spi")
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv(f"../public/island_spi{scale}.csv")
print(df)


date        2024-09   2024-10   2024-11   2024-12   2025-01   2025-02  \
division                                                                
Hawaiʻi   -0.037881  0.119707  0.156255  0.029552  0.153153  0.034498   
Kahoolawe -0.139777 -0.032876 -0.204780 -0.281347 -0.506336 -0.530695   
Kauaʻi     0.296000  0.232871 -0.212814 -0.599307 -0.768473 -0.859197   
Lānaʻi    -0.126588 -0.045729 -0.234460 -0.373994 -0.823798 -0.897692   
Maui      -0.508126 -0.303986 -0.468384 -0.689602 -0.970109 -1.112548   
Molokaʻi  -0.215129 -0.137697 -0.291784 -0.514934 -0.797169 -0.941964   
Oʻahu      0.251069  0.466929  0.185043 -0.173402 -0.133351 -0.257141   

date        2025-03   2025-04   2025-05   2025-06   2025-07   2025-08  
division                                                               
Hawaiʻi    0.190834  0.182623 -0.052502 -0.007167 -0.046251 -0.683822  
Kahoolawe -0.437860 -0.150719 -0.209612 -0.212079 -0.206543 -0.320421  
Kauaʻi    -0.639868 -1.425405 -1.558343 -1.472391 -1.3

In [23]:
#statewide
shapefile = "../public/Coastline.shp"
raster_folder = "./data"

# Load shapefile and dissolve to one feature (statewide)
gdf = gpd.read_file(shapefile)
gdf_statewide = gdf.dissolve()  # merges all polygons into one

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, f"spi{scale:03d}*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Compute zonal stats for the dissolved geometry
    stats = zonal_stats(
        vectors=gdf_statewide,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    records.append({
        "date": date,
        "value": stats[0]["mean"]
    })


# Pivot to wide format: one row (statewide), months as columns
df = pd.DataFrame(records).set_index("date").T
df.insert(0, "state", "statewide")  # add proper state column
df.to_csv(f"../public/statewide_spi{scale}.csv", index=False)

print(df)

date       state   2024-09   2024-10   2024-11   2024-12   2025-01   2025-02  \
value  statewide -0.045008  0.098565  0.024834 -0.161592 -0.150209 -0.269035   

date    2025-03   2025-04   2025-05   2025-06  2025-07   2025-08  
value -0.095722 -0.171698 -0.439413 -0.401593 -0.40669 -0.875027  
